In [16]:
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency

In [17]:
##import player_data.csv file and build dataframe##
df_player_data = pd.read_csv("player_data.csv")
df_player_data.drop('Unnamed: 0',axis='columns', inplace=True)
df_player_data.drop('joined_date',axis='columns', inplace=True)

In [18]:
##import player_list.csv file and build dataframe##
df_player_list = pd.read_csv("player_list.csv")

In [19]:
##addition of the "player name" from column df_player_list daaframe to df_player_data dataframe##
df_player_data = pd.concat([df_player_list['player name'],df_player_data], axis=1)


In [20]:
##Clearing the data from unwanted marks
##Convert players price to a number format
##Changing data types (from objects to int/float)
df_player_data['highest value-EUR']=df_player_data['highest value-EUR'].map(lambda x: x.lstrip('€')).str.replace('m','0000')
df_player_data['current value-EUR']=df_player_data['current value-EUR'].map(lambda x: x.lstrip('€')).str.replace('m','0000')
df_player_data['highest value-EUR']=df_player_data['highest value-EUR'].str.replace('Th.','000').str.replace('.','').astype(int)
df_player_data['current value-EUR']=df_player_data['current value-EUR'].str.replace('Th.','000').str.replace('.','').astype(int)


In [21]:
##Set dictionary for "foot" column
##Replace "NaN" values with '0' 
df_player_data['foot'] = df_player_data['foot'].fillna(0)
replace_foot = {'left':1, 'right':2, 'both':3}
df_player_data.replace(replace_foot, inplace=True)
df_player_data['foot'] = df_player_data['foot'].astype(int)


In [22]:
##Clearing the data from unwanted marks
##Changing data types (from objects to int/float)
df_player_data['height'] = df_player_data['height'].str.replace(',','.').astype(float)

In [23]:
##Sorts the dataframe according to the ages of the players
df_player_data.sort_values(by=['age'], ascending=False, inplace=True)
df_player_data.reset_index(drop=True, inplace=True)

In [24]:
df_player_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8213 entries, 0 to 8212
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player name        8213 non-null   object 
 1   date_of_birth      8213 non-null   object 
 2   place_of_birth     7802 non-null   object 
 3   age                8213 non-null   int64  
 4   height             8110 non-null   float64
 5   citizenship        8213 non-null   object 
 6   position           8213 non-null   object 
 7   foot               8213 non-null   int32  
 8   current_club       8213 non-null   object 
 9   current value-EUR  8213 non-null   int32  
 10  highest value-EUR  8213 non-null   int32  
 11  first_club         8213 non-null   object 
 12  transfer_count     8213 non-null   int64  
dtypes: float64(1), int32(3), int64(2), object(7)
memory usage: 738.0+ KB


In [25]:
df_player_data.isna().sum()

player name            0
date_of_birth          0
place_of_birth       411
age                    0
height               103
citizenship            0
position               0
foot                   0
current_club           0
current value-EUR      0
highest value-EUR      0
first_club             0
transfer_count         0
dtype: int64

In [26]:
##For empty values in height column(float) the following code replace 'NaN' with the median,##
##and "place_of_birth" column(object) with "unknown" string                                 ##
numeric_col = df_player_data._get_numeric_data().columns
numeric=df_player_data[numeric_col].apply(lambda col: col.fillna(col.median()), axis=0)

non_numeric_col= list(filter(lambda col: col not in numeric_col, df_player_data.columns))
non_numeric = df_player_data[non_numeric_col].apply(lambda col: col.fillna("unknown"), axis=0)

df_player_data = pd.concat((numeric,non_numeric), axis=1, sort=False)


sequence = ['player name','age','citizenship','first_club','current_club','date_of_birth','place_of_birth','position','height',
            'foot', 'transfer_count','current value-EUR','highest value-EUR']
df_player_data = df_player_data.reindex(columns=sequence)

In [27]:
df_player_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8213 entries, 0 to 8212
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player name        8213 non-null   object 
 1   age                8213 non-null   int64  
 2   citizenship        8213 non-null   object 
 3   first_club         8213 non-null   object 
 4   current_club       8213 non-null   object 
 5   date_of_birth      8213 non-null   object 
 6   place_of_birth     8213 non-null   object 
 7   position           8213 non-null   object 
 8   height             8213 non-null   float64
 9   foot               8213 non-null   int32  
 10  transfer_count     8213 non-null   int64  
 11  current value-EUR  8213 non-null   int32  
 12  highest value-EUR  8213 non-null   int32  
dtypes: float64(1), int32(3), int64(2), object(7)
memory usage: 738.0+ KB


In [28]:
# df_21_and_under = df_player_data[df_player_data['age'] <= 21]
# df_over_21 = df_player_data[df_player_data['age'] > 21]

In [29]:
df_player_data

,player name,age,citizenship,first_club,current_club,date_of_birth,place_of_birth,position,height,foot,transfer_count,current value-EUR,highest value-EUR
0,Joaquín,39,Spain,Betis B,Real Betis Balompié,"Jul 21, 1981",El Puerto de Santa María,attack - Right Winger,1.81,2,6,1500000,28000000
1,Jorge Molina,39,Spain,CD Alcoyano,Granada CF,"Apr 22, 1982",Alcoi,attack - Centre-Forward,1.88,2,8,1500000,3500000
2,Leonardo Burián,37,Uruguay,Nacional,Club Atlético Colón,"Jan 21, 1984",Melo,Goalkeeper,1.87,2,11,1100000,1100000
3,Denis Onyango,36,Uganda South Africa,Saint George SA,Mamelodi Sundowns FC,"May 15, 1985",Kampala,Goalkeeper,1.85,3,6,750000,1000000
4,Sebastián Tagliabúe,36,United Arab Emirates Argentina,Colegiales,Al-Nasr (Dubai),"Feb 22, 1985",Olivos,attack - Centre-Forward,1.81,2,8,800000,2000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8208,Ângelo Gabriel,16,Brazil,Santos FC,Santos FC,"Dec 21, 2004",Brasília,attack - Right Winger,1.73,1,1,7000000,7000000
8209,Luka Romero,16,Argentina Spain,PE Sant Jordi,RCD Mallorca B,"Nov 18, 2004",Victoria de Durango,midfield - Attacking Midfield,1.65,2,4,1000000,1000000
8210,Noah Mbamba,16,Belgium,FC Brügge U18,Club Brugge KV,"Jan 5, 2005",Elsene,Defender - Centre-Back,1.82,2,3,800000,800000
8211,Sergey Pinyaev,16,Russia,Akad Chertanovo,Chertanovo Moscow,"Nov 2, 2004",Saratov,attack - Left Winger,1.67,0,4,800000,800000


In [30]:
df_player_data[numeric_col].corr()

,age,height,foot,current value-EUR,highest value-EUR,transfer_count
age,1.000000,0.074214,0.041357,-0.041407,0.001954,0.394695
height,0.074214,1.000000,0.026946,0.039252,0.033837,0.087707
foot,0.041357,0.026946,1.000000,0.043127,0.044147,0.023445
current value-EUR,-0.041407,0.039252,0.043127,1.000000,0.952720,-0.004835
highest value-EUR,0.001954,0.033837,0.044147,0.952720,1.000000,0.016895
transfer_count,0.394695,0.087707,0.023445,-0.004835,0.016895,1.000000


In [ ]:
##chi2 tests:

In [ ]:
def encode(data, columns):
    encoders = {}
    reverse_encoders = {}
    for col in columns:
        encoder = LabelEncoder()
        data[col] = encoder.fit_transform(data[col])
        encoder = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
        reverse_encoder = dict(map(reversed, encoder.items()))
        encoders[col] = encoder
        reverse_encoders[col] = reverse_encoder
    return data[columns], encoders, reverse_encoders

encoded_data, encoders, reverse_encoders = encode(df_player_data.copy(), non_numeric_col)

In [ ]:
encoded_data

In [ ]:
seen_pairs = []
results = []
for col_a in encoded_data.columns:
    for col_b in tqdm(encoded_data.columns):
        if sorted((col_a, col_b)) in seen_pairs:
            continue
        results.append((col_a, col_b, chi2_contingency(pd.crosstab(encoded_data[col_a],encoded_data[col_b]))[:3]))
        break
    break

In [ ]:
chi2_test = pd.DataFrame(results,columns = ['A', 'B', 'results'])
chi2_test = pd.concat((chi2_test.drop('results', axis=1), chi2_test['results'].apply(pd.Series)),axis=1)
chi2_test.columns = ['A', 'B', 'chi2' , 'p', 'dof']
chi2_test

In [ ]:
##Export the dataframe to CSV file###
df_player_data.to_csv("player_data_updated.csv")